This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Static clutter map

Detects the average level of the returned signal. It is recommended to use at least one month of data.

In [ ]:
try:
    %load_ext line_profiler
    %load_ext memory_profiler
except:
    pass


import glob
import os
import warnings

import numpy as np
import matplotlib
import matplotlib.pyplot as pl
warnings.filterwarnings('ignore')
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()

import wradlib as wrl
import wradlib.clutter as clutter
import wradlib.util as util
import wradlib.vis as vis

In [ ]:
import sys
import os
import psutil
process = psutil.Process(os.getpid())

### Load sweep data

In [ ]:
wrl_path = util.get_wradlib_data_path()
path = os.path.join(wrl_path,'hdf5/bejab/*dB*')

files = glob.glob(path)
files.sort()
#files = files[0:10]
#print(files)

pvol = wrl.io.xarray.open_odim(files,
                               parallel=True,
                               mask_and_scale=True,
                               loader="h5py",
                               chunks={}
                              )
print(process.memory_info().rss)


### Select and decode data

In [ ]:
sweep = pvol[-1]
print(process.memory_info().rss)
elangle = str(sweep[0].fixed_angle)
print(elangle)
sweep = sweep.data
print(sweep)
print(process.memory_info().rss)
maxdist = 100
sweep = sweep.sel(range=slice(0,maxdist*1E3))
startdate = str(sweep.time.values[0])[:10]
enddate = str(sweep.time.values[-1])[:10]


print(process.memory_info().rss)



### Restore values removed by CSR (optional)


### Get the static clutter levels

In [ ]:
import importlib
importlib.reload(clutter)
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

import time

var = "DBZH"
#var = "TH"
start = time.time()
cmap = clutter.static_clutter_map(sweep,
                                   var=var,
#                                   csr=10
                                   )
print(time.time()-start)
print(process.memory_info().rss)

### Plot the results

In [ ]:
cmap = cmap.pipe(wrl.georef.georeference_dataset)

source = pvol.what["source"]
source = source.split("NOD:")[1][0:5]

pl.rcParams.update({'font.size': 16})

datestr = "{}-{}".format(startdate, enddate)
diststr = "{}km".format(maxdist)

params = [source, var, elangle, diststr, datestr]
titlestr = " ".join(params)

size = 12

da = cmap["static_clutter_level"]


fig = pl.figure(figsize=(size*4/3,size))
fig.suptitle(titlestr)
pl.axis("off")
pm = da.wradlib.plot_ppi(fig=fig, vmin=0, vmax=60)
ax = pl.gca()
cb = pl.colorbar(pm, ax=ax, label="static_clutter_level")
filename = "_".join(params)
filename = "{}_static_clutter_level.png".format(filename)
pl.savefig(filename, bbox_inches='tight', format="png")

da = cmap["static_clutter_width"]
fig = pl.figure(figsize=(size*4/3,size))    
fig.suptitle(titlestr)
pl.axis("off")
pm = da.wradlib.plot_ppi(fig=fig, vmin=0, vmax=20)
ax = pl.gca()
cb = pl.colorbar(pm, ax=ax, label="static_clutter_width")
filename = "_".join(params)
filename = "{}_static_clutter_width.png".format(filename)
pl.savefig(filename, bbox_inches='tight')
        
        